In [1]:
def myLoadImage(im_path,img_size, resize=False, augment=True) :
    im = cv2.imread(im_path)
    print(im.shape)
    h0, w0 = im.shape[:2]
    if resize : 
        r = img_size / max(h0, w0)
        if r != 1:  # if sizes are not equal
            interp = cv2.INTER_LINEAR if augment else cv2.INTER_AREA  # random.choice(self.rand_interp_methods)
            im = cv2.resize(im, (int(w0 * r), int(h0 * r)), interpolation=interp)
    return im, (h0, w0), im.shape[:2] 

from PIL import Image, ImageDraw
import matplotlib.patches as patches
def myFig(img, bbox_list, drawB = True) :
    image = Image.fromarray(img,"RGB")
    draw = ImageDraw.Draw(image)

    if drawB :
        for bbox in bbox_list : 
            for i in range(len(bbox)) :
                xmin = bbox[i][1] 
                ymin = bbox[i][2]
                xmax = bbox[i][3]
                ymax = bbox[i][4]

                draw.rectangle((xmin, ymin, xmax, ymax), outline=(255,0,0), width = 1) # bounding box
#     display(image)
    return image
def xywhn2xyxy(x, w=640, h=640, padw=0, padh=0):
    # Convert nx4 boxes from [x, y, w, h] normalized to [x1, y1, x2, y2] where xy1=top-left, xy2=bottom-right
    y = x.clone() if isinstance(x, torch.Tensor) else np.copy(x)
    y[:, 0] = w * (x[:, 0] - x[:, 2] / 2) + padw  # top left x
    y[:, 1] = h * (x[:, 1] - x[:, 3] / 2) + padh  # top left y
    y[:, 2] = w * (x[:, 0] + x[:, 2] / 2) + padw  # bottom right x
    y[:, 3] = h * (x[:, 1] + x[:, 3] / 2) + padh  # bottom right y
    return y

def convert_coordinates(x1, y1, x2, y2, img_width, img_height, cate):
    # 바운딩 박스의 중심 좌표와 너비, 높이 계산
    box_width = x2 - x1
    box_height = y2 - y1
    x_center = (x1 + x2) / 2.0
    y_center = (y1 + y2) / 2.0
    
    # 이미지의 너비와 높이로 나누어서 정규화된 값 계산
    x_center /= img_width
    y_center /= img_height
    box_width /= img_width
    box_height /= img_height
    
    # 결과를 문자열로 반환
    return f"{cate} {x_center:.6f} {y_center:.6f} {box_width:.6f} {box_height:.6f}"

In [2]:
img_size = 256
category_id = 1

def test(img_list, save_img_path, save_label_path, save_annotations_path,coco_json,id_cnt,t):
    
    for i, imgs in enumerate(img_list) : 
        result_img = {}
        for idx , img_path in enumerate(imgs) : 
            img, (h0, w0), (h, w) = myLoadImage(img_path,img_size)

            color = random.randrange(0,255)
            back_ground = np.full((img_size * 2, img_size * 2, img.shape[2]), color, dtype=np.uint8) 
            mosaic_border = [img_size // 2, img_size // 2]

            yc, xc = (int(random.uniform(img_size, 2 *img_size)) for x in mosaic_border)  

            x1a, y1a, x2a, y2a = max(xc - w, 0), max(yc - h, 0), xc, yc  # xmin, ymin, xmax, ymax (large image)
            x1b, y1b, x2b, y2b = w - (x2a - x1a), h - (y2a - y1a), w, h  # xmin, ymin, xmax, ymax (small image)
            xmin, ymin = x1a, y1a

            if int(x2a - x1a)  < img_size : 
                print(f'size ERROR')
            if int(y2a - y1a)  < img_size : 
                print(f'size ERROR')
            back_ground[y1a:y2a, x1a:x2a] = img[y1b:y2b, x1b:x2b]  # img4[ymin:ymax, xmin:xmax]

            padw = x1a - x1b
            padh = y1a - y1b

            label = np.array([[0. , 0.5 ,0.5 ,1. , 1. ]])

            if label.size:
                label[:, 1:] = xywhn2xyxy(label[:, 1:], w, h, padw, padh)  # normalized xywh to pixel xyxy format
                label[label < 0] = 0
                label[label > img_size*2] = img_size*2

            if idx == 0 : 
                pass
            elif idx == 1 :
                label[0][1] += img_size * 2
                label[0][3] += img_size * 2
            elif idx == 2 :
                label[0][2] += img_size * 2
                label[0][4] += img_size * 2
            elif idx == 3 :
                label[0][1] += img_size * 2
                label[0][2] += img_size * 2
                label[0][3] += img_size * 2
                label[0][4] += img_size * 2

            result_img[idx] = {'img' : back_ground, 'label':label}

        if len(result_img) ==4 : 

            res_img1 = np.concatenate((result_img[0]['img'],result_img[1]['img']),axis=1)
            res_img2 = np.concatenate((result_img[2]['img'],result_img[3]['img']),axis=1)
            result_image = np.concatenate((res_img1,res_img2),axis=0)
            label_list = [result_img[idx]['label'] for idx in result_img ]
        else :
            result_image = np.concatenate((result_img[0]['img'],result_img[1]['img']),axis=1)
            label_list = [result_img[idx]['label'] for idx in result_img ]

        cv2.imwrite(f'{save_img_path}/{i+t}.jpg',result_image)
        height, width, _ = result_image.shape

        img_res = {'license': 1, 'file_name': f'{i+t}.jpg','coco_url': '','height': height,'width': width,'date_captured': '','flickr_url': '','id': i+t}

        coco_json['images'].append(img_res)

        with open(f'{save_label_path}/{i+t}.txt', "w") as f:
            for idx in result_img : 
                label = result_img[idx]['label'][0]

                x1, y1, x2, y2 = label[1],label[2],label[3],label[4]

                yolo_bbox = convert_coordinates(x1, y1, x2, y2, width, height, category_id)
                f.write(f"{yolo_bbox}\n")

                coco_bbox = [x1, y1, x2 - x1, y2 - y1]  # COCO bbox 형식으로 변환

                area = coco_bbox[2] * coco_bbox[3]
                res = {'segmentation':[] ,'area': area ,'iscrowd': 0,'image_id': i+t,'bbox': coco_bbox ,'category_id': category_id,'id': id_cnt}
                coco_json['annotations'].append(res)
                id_cnt += 1
                print(res)


    import json
    with open(save_annotations_path, "w") as f:
        json.dump(coco_json, f)
    return id_cnt



In [3]:
import math
import os
import random
from pathlib import Path
import numpy as np
import yaml
import glob
import cv2
import torch
from pathlib import Path
import pandas as pd
file_ex = "png"
path = '/data/ij/Mosaic_data_augmentation_YOLO/logitech'
# path = '/data/ij/Mosaic_data_augmentation_YOLO/images'
save_img_path = '/data/ij/Mosaic_data_augmentation_YOLO/coco_cp/images/train2017'
save_img_path_val = '/data/ij/Mosaic_data_augmentation_YOLO/coco_cp/images/val2017'

save_label_path = '/data/ij/Mosaic_data_augmentation_YOLO/coco_cp/labels/train2017'
save_label_path_val = '/data/ij/Mosaic_data_augmentation_YOLO/coco_cp/labels/val2017'

save_annotations_path = '/data/ij/Mosaic_data_augmentation_YOLO/coco_cp/annotations/instances_train2017.json'
save_annotations_path_val = '/data/ij/Mosaic_data_augmentation_YOLO/coco_cp/annotations/instances_val2017.json'

im_files = np.array([i for i in os.listdir(path) if i.endswith(file_ex)], dtype = "object")
im_files = list(path + "/" + im_files)

In [4]:
im_f_size = len(im_files)

img_list = []
for div in range(4, im_f_size, 4) :

    img_list.append(im_files[div-4:div])

img_list.append(im_files[len(img_list) * 4:])
img_list

[['/data/ij/Mosaic_data_augmentation_YOLO/logitech/generated_image_573.png',
  '/data/ij/Mosaic_data_augmentation_YOLO/logitech/generated_image_473.png',
  '/data/ij/Mosaic_data_augmentation_YOLO/logitech/generated_image_287.png',
  '/data/ij/Mosaic_data_augmentation_YOLO/logitech/generated_image_327.png'],
 ['/data/ij/Mosaic_data_augmentation_YOLO/logitech/generated_image_541.png',
  '/data/ij/Mosaic_data_augmentation_YOLO/logitech/generated_image_393.png',
  '/data/ij/Mosaic_data_augmentation_YOLO/logitech/generated_image_559.png',
  '/data/ij/Mosaic_data_augmentation_YOLO/logitech/generated_image_274.png'],
 ['/data/ij/Mosaic_data_augmentation_YOLO/logitech/generated_image_566.png',
  '/data/ij/Mosaic_data_augmentation_YOLO/logitech/generated_image_392.png',
  '/data/ij/Mosaic_data_augmentation_YOLO/logitech/generated_image_530.png',
  '/data/ij/Mosaic_data_augmentation_YOLO/logitech/generated_image_579.png'],
 ['/data/ij/Mosaic_data_augmentation_YOLO/logitech/generated_image_280.pn

In [5]:
split = int(len(img_list) * 0.8)
train_img_list = img_list[:split]
val_img_list = img_list[split:]

In [6]:
coco_json_train = {}
coco_json_train['licenses'] = [{'url': 'http://creativecommons.org/licenses/by-nc-sa/2.0/','id': 1,'name': 'Attribution-NonCommercial-ShareAlike License'}]
coco_json_train['info'] = {'description': 'COCO apple Dataset','url': '','version': '','year': 2023,'contributor': 'COCO Consortium','date_created': '2023/04/11'}
coco_json_train['annotations'] = []
coco_json_train['categories'] = [{'supercategory': 'logitech', 'id': 1, 'name': 'logitech'}]
coco_json_train['images'] = []


coco_json_val = {}
coco_json_val['licenses'] = [{'url': 'http://creativecommons.org/licenses/by-nc-sa/2.0/','id': 1,'name': 'Attribution-NonCommercial-ShareAlike License'}]
coco_json_val['info'] = {'description': 'COCO apple Dataset','url': '','version': '','year': 2023,'contributor': 'COCO Consortium','date_created': '2023/04/11'}
coco_json_val['annotations'] = []
coco_json_val['categories'] = [{'supercategory': 'logitech', 'id': 1, 'name': 'logitech'}]
coco_json_val['images'] = []
id_cnt = 1
t = 0
id_cnt= test(train_img_list, save_img_path, save_label_path, save_annotations_path,coco_json_train, id_cnt,t)
t = int(len(train_img_list))
id_cnt= test(val_img_list, save_img_path_val, save_label_path_val, save_annotations_path_val,coco_json_val, id_cnt,t)


(256, 256, 3)
(256, 256, 3)
(256, 256, 3)
(256, 256, 3)
{'segmentation': [], 'area': 65536.0, 'iscrowd': 0, 'image_id': 0, 'bbox': [64.0, 157.0, 256.0, 256.0], 'category_id': 1, 'id': 1}
{'segmentation': [], 'area': 65536.0, 'iscrowd': 0, 'image_id': 0, 'bbox': [713.0, 19.0, 256.0, 256.0], 'category_id': 1, 'id': 2}
{'segmentation': [], 'area': 65536.0, 'iscrowd': 0, 'image_id': 0, 'bbox': [113.0, 595.0, 256.0, 256.0], 'category_id': 1, 'id': 3}
{'segmentation': [], 'area': 65536.0, 'iscrowd': 0, 'image_id': 0, 'bbox': [634.0, 730.0, 256.0, 256.0], 'category_id': 1, 'id': 4}
(256, 256, 3)
(256, 256, 3)
(256, 256, 3)
(256, 256, 3)
{'segmentation': [], 'area': 65536.0, 'iscrowd': 0, 'image_id': 1, 'bbox': [60.0, 214.0, 256.0, 256.0], 'category_id': 1, 'id': 5}
{'segmentation': [], 'area': 65536.0, 'iscrowd': 0, 'image_id': 1, 'bbox': [623.0, 33.0, 256.0, 256.0], 'category_id': 1, 'id': 6}
{'segmentation': [], 'area': 65536.0, 'iscrowd': 0, 'image_id': 1, 'bbox': [221.0, 554.0, 256.0, 256

(256, 256, 3)
(256, 256, 3)
{'segmentation': [], 'area': 65536.0, 'iscrowd': 0, 'image_id': 15, 'bbox': [106.0, 202.0, 256.0, 256.0], 'category_id': 1, 'id': 61}
{'segmentation': [], 'area': 65536.0, 'iscrowd': 0, 'image_id': 15, 'bbox': [586.0, 20.0, 256.0, 256.0], 'category_id': 1, 'id': 62}
{'segmentation': [], 'area': 65536.0, 'iscrowd': 0, 'image_id': 15, 'bbox': [212.0, 735.0, 256.0, 256.0], 'category_id': 1, 'id': 63}
{'segmentation': [], 'area': 65536.0, 'iscrowd': 0, 'image_id': 15, 'bbox': [702.0, 521.0, 256.0, 256.0], 'category_id': 1, 'id': 64}
(256, 256, 3)
(256, 256, 3)
(256, 256, 3)
(256, 256, 3)
{'segmentation': [], 'area': 65536.0, 'iscrowd': 0, 'image_id': 16, 'bbox': [93.0, 92.0, 256.0, 256.0], 'category_id': 1, 'id': 65}
{'segmentation': [], 'area': 65536.0, 'iscrowd': 0, 'image_id': 16, 'bbox': [746.0, 173.0, 256.0, 256.0], 'category_id': 1, 'id': 66}
{'segmentation': [], 'area': 65536.0, 'iscrowd': 0, 'image_id': 16, 'bbox': [131.0, 514.0, 256.0, 256.0], 'category

{'segmentation': [], 'area': 65536.0, 'iscrowd': 0, 'image_id': 31, 'bbox': [208.0, 234.0, 256.0, 256.0], 'category_id': 1, 'id': 125}
{'segmentation': [], 'area': 65536.0, 'iscrowd': 0, 'image_id': 31, 'bbox': [735.0, 97.0, 256.0, 256.0], 'category_id': 1, 'id': 126}
{'segmentation': [], 'area': 65536.0, 'iscrowd': 0, 'image_id': 31, 'bbox': [46.0, 587.0, 256.0, 256.0], 'category_id': 1, 'id': 127}
{'segmentation': [], 'area': 65536.0, 'iscrowd': 0, 'image_id': 31, 'bbox': [629.0, 719.0, 256.0, 256.0], 'category_id': 1, 'id': 128}
(256, 256, 3)
(256, 256, 3)
(256, 256, 3)
(256, 256, 3)
{'segmentation': [], 'area': 65536.0, 'iscrowd': 0, 'image_id': 32, 'bbox': [141.0, 156.0, 256.0, 256.0], 'category_id': 1, 'id': 129}
{'segmentation': [], 'area': 65536.0, 'iscrowd': 0, 'image_id': 32, 'bbox': [713.0, 0.0, 256.0, 256.0], 'category_id': 1, 'id': 130}
{'segmentation': [], 'area': 65536.0, 'iscrowd': 0, 'image_id': 32, 'bbox': [245.0, 766.0, 256.0, 256.0], 'category_id': 1, 'id': 131}
{'s

(256, 256, 3)
(256, 256, 3)
(256, 256, 3)
{'segmentation': [], 'area': 65536.0, 'iscrowd': 0, 'image_id': 48, 'bbox': [207.0, 114.0, 256.0, 256.0], 'category_id': 1, 'id': 193}
{'segmentation': [], 'area': 65536.0, 'iscrowd': 0, 'image_id': 48, 'bbox': [697.0, 109.0, 256.0, 256.0], 'category_id': 1, 'id': 194}
{'segmentation': [], 'area': 65536.0, 'iscrowd': 0, 'image_id': 48, 'bbox': [117.0, 591.0, 256.0, 256.0], 'category_id': 1, 'id': 195}
{'segmentation': [], 'area': 65536.0, 'iscrowd': 0, 'image_id': 48, 'bbox': [677.0, 696.0, 256.0, 256.0], 'category_id': 1, 'id': 196}
(256, 256, 3)
(256, 256, 3)
(256, 256, 3)
(256, 256, 3)
{'segmentation': [], 'area': 65536.0, 'iscrowd': 0, 'image_id': 49, 'bbox': [74.0, 206.0, 256.0, 256.0], 'category_id': 1, 'id': 197}
{'segmentation': [], 'area': 65536.0, 'iscrowd': 0, 'image_id': 49, 'bbox': [550.0, 50.0, 256.0, 256.0], 'category_id': 1, 'id': 198}
{'segmentation': [], 'area': 65536.0, 'iscrowd': 0, 'image_id': 49, 'bbox': [93.0, 747.0, 256.

{'segmentation': [], 'area': 65536.0, 'iscrowd': 0, 'image_id': 67, 'bbox': [193.0, 65.0, 256.0, 256.0], 'category_id': 1, 'id': 269}
{'segmentation': [], 'area': 65536.0, 'iscrowd': 0, 'image_id': 67, 'bbox': [566.0, 181.0, 256.0, 256.0], 'category_id': 1, 'id': 270}
{'segmentation': [], 'area': 65536.0, 'iscrowd': 0, 'image_id': 67, 'bbox': [77.0, 567.0, 256.0, 256.0], 'category_id': 1, 'id': 271}
{'segmentation': [], 'area': 65536.0, 'iscrowd': 0, 'image_id': 67, 'bbox': [594.0, 624.0, 256.0, 256.0], 'category_id': 1, 'id': 272}
(256, 256, 3)
(256, 256, 3)
(256, 256, 3)
(256, 256, 3)
{'segmentation': [], 'area': 65536.0, 'iscrowd': 0, 'image_id': 68, 'bbox': [133.0, 47.0, 256.0, 256.0], 'category_id': 1, 'id': 273}
{'segmentation': [], 'area': 65536.0, 'iscrowd': 0, 'image_id': 68, 'bbox': [761.0, 238.0, 256.0, 256.0], 'category_id': 1, 'id': 274}
{'segmentation': [], 'area': 65536.0, 'iscrowd': 0, 'image_id': 68, 'bbox': [87.0, 716.0, 256.0, 256.0], 'category_id': 1, 'id': 275}
{'s

{'segmentation': [], 'area': 65536.0, 'iscrowd': 0, 'image_id': 85, 'bbox': [203.0, 7.0, 256.0, 256.0], 'category_id': 1, 'id': 341}
{'segmentation': [], 'area': 65536.0, 'iscrowd': 0, 'image_id': 85, 'bbox': [764.0, 107.0, 256.0, 256.0], 'category_id': 1, 'id': 342}
{'segmentation': [], 'area': 65536.0, 'iscrowd': 0, 'image_id': 85, 'bbox': [26.0, 676.0, 256.0, 256.0], 'category_id': 1, 'id': 343}
{'segmentation': [], 'area': 65536.0, 'iscrowd': 0, 'image_id': 85, 'bbox': [600.0, 534.0, 256.0, 256.0], 'category_id': 1, 'id': 344}
(256, 256, 3)
(256, 256, 3)
(256, 256, 3)
(256, 256, 3)
{'segmentation': [], 'area': 65536.0, 'iscrowd': 0, 'image_id': 86, 'bbox': [51.0, 221.0, 256.0, 256.0], 'category_id': 1, 'id': 345}
{'segmentation': [], 'area': 65536.0, 'iscrowd': 0, 'image_id': 86, 'bbox': [616.0, 170.0, 256.0, 256.0], 'category_id': 1, 'id': 346}
{'segmentation': [], 'area': 65536.0, 'iscrowd': 0, 'image_id': 86, 'bbox': [243.0, 593.0, 256.0, 256.0], 'category_id': 1, 'id': 347}
{'s

{'segmentation': [], 'area': 65536.0, 'iscrowd': 0, 'image_id': 101, 'bbox': [122.0, 46.0, 256.0, 256.0], 'category_id': 1, 'id': 405}
{'segmentation': [], 'area': 65536.0, 'iscrowd': 0, 'image_id': 101, 'bbox': [709.0, 25.0, 256.0, 256.0], 'category_id': 1, 'id': 406}
{'segmentation': [], 'area': 65536.0, 'iscrowd': 0, 'image_id': 101, 'bbox': [17.0, 736.0, 256.0, 256.0], 'category_id': 1, 'id': 407}
{'segmentation': [], 'area': 65536.0, 'iscrowd': 0, 'image_id': 101, 'bbox': [674.0, 554.0, 256.0, 256.0], 'category_id': 1, 'id': 408}
(256, 256, 3)
(256, 256, 3)
(256, 256, 3)
(256, 256, 3)
{'segmentation': [], 'area': 65536.0, 'iscrowd': 0, 'image_id': 102, 'bbox': [211.0, 159.0, 256.0, 256.0], 'category_id': 1, 'id': 409}
{'segmentation': [], 'area': 65536.0, 'iscrowd': 0, 'image_id': 102, 'bbox': [544.0, 61.0, 256.0, 256.0], 'category_id': 1, 'id': 410}
{'segmentation': [], 'area': 65536.0, 'iscrowd': 0, 'image_id': 102, 'bbox': [62.0, 689.0, 256.0, 256.0], 'category_id': 1, 'id': 41

In [7]:
myFig(cv2.cvtColor(result_image, cv2.COLOR_BGR2RGB), label_list)

NameError: name 'result_image' is not defined

In [ ]:
coco_json['images']

In [ ]:
for i in result_img : 
    print(i)